In [1]:
import pandas as pd
import numpy as np
import json, requests
import time
import glob
import re
import copy

In [2]:
mappings = pd.read_csv("cleaned_mappings.csv")
mappings.head()

,Faculty,Partner University,PU Module 1,PU Module 1 Title,PU Mod1 Credits,PU Module 2,PU Module 2 Title,PU Mod2 Credits,NUS Module 1,NUS Module 1 Title,NUS Mod1 Credits,NUS Module 2,NUS Module 2 Title,NUS Mod2 Credits,Pre Approved?
0,Faculty of Arts & Social Sci,The Hong Kong Polytechnic University,CBS241,Elementary Chinese II (for Non-Chinese speakin...,1.0,NaN,NaN,NaN,LAC2731,Department Exchange Module,3.0,NaN,NaN,NaN,Y
1,Faculty of Arts & Social Sci,The Hong Kong Polytechnic University,CC2C08,Mutual Impressions of China and the West,3.0,NaN,NaN,NaN,PS2238,Int'l Politics of NE Asia,4.0,NaN,NaN,NaN,Y
2,Faculty of Arts & Social Sci,Hong Kong University of Science & Technology,LANG1120,Chinese for Non-Chinese Language Background St...,1.0,NaN,NaN,NaN,LAC1731,Department exchange module,3.0,NaN,NaN,NaN,Y
3,Faculty of Arts & Social Sci,City University of Hong Kong,AIS3126,International Political Economy,3.0,NaN,NaN,NaN,PS3238,Int'l Political Economy,4.0,NaN,NaN,NaN,Y
4,Faculty of Arts & Social Sci,City University of Hong Kong,GE2210,China: A Socio-Political Transformation,3.0,NaN,NaN,NaN,PS2248,Chinese Politics,4.0,NaN,NaN,NaN,Y


In [33]:
def get_department(title):
    end_index = 0
    for i, char in enumerate(title):
        if not char.isalpha():
            end_index = i
            break
    return title[:end_index]

def get_equivalent_modules(modules):
    return modules
    
def department_filter(departments, mappings=mappings):
    if len(departments) == 0:
        return mappings

    filter_function = lambda title : (type(title) == str) and (get_department(title) in departments)
    return mappings[mappings['NUS Module 1'].map(filter_function) | mappings['NUS Module 2'].map(filter_function)]


def module_filter(modules, mappings=mappings):
    if len(modules) == 0:
        return mappings
    
    equivalent_modules = get_equivalent_modules(modules)
    filter_function = lambda title: title in equivalent_modules
    return mappings[mappings['NUS Module 1'].map(filter_function) | mappings['NUS Module 2'].map(filter_function)]

def school_filter(schools, mappings=mappings):
    if len(schools) == 0:
        return mappings
    
    filter_function = lambda school: school in schools
    return mappings[mappings['Partner University'].map(filter_function)]

In [39]:
all_schools

{'Aalto University',
 'Aalto University, Helsinki',
 'Aalto University, Mikkeli',
 'Aarhus School of Business',
 'Aarhus University',
 'Amsterdam University College',
 'Arizona State University',
 'Athens University of Economics & Business',
 'BI Norwegian Business School',
 'Beihang University',
 'Bilkent University',
 'Bogazici University, Turkey',
 'Boston College, Massachusetts',
 'Boston University',
 'Brandeis University',
 'Bremen University of Applied Sciences',
 'Budapest University of Technology and Economics',
 'California Poly State Uni, SLO',
 'California State University East bay',
 'Carnegie Mellon University',
 'Case Western Reserve University',
 'Catholic University of Portugal',
 'Central University of Finance & Economics',
 'CentraleSupelec - Ecole Centrale Paris',
 'Chalmers University of Technology',
 'Chang Gung University',
 'Charles University in Prague',
 'Chongqing University',
 'Chulalongkorn University',
 'City University of Hong Kong',
 'City, University of

In [38]:
all_schools = set(mappings['Partner University'].dropna().unique())
all_departments = set(mappings['NUS Module 1'].map(get_department).unique()).union(set(mappings['NUS Module 1'].dropna().map(get_department).unique()))
all_modules = set(mappings['NUS Module 1'].dropna().unique()).union(set(mappings['NUS Module 2'].dropna().unique()))

In [36]:
%timeit school_filter(all_schools, module_filter(all_modules, department_filter(all_departments)))

20.8 ms ± 53.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
